# Project Iteration

Real code projects change over time as developers add features, refactor code to improve organization, and write tests to make sure their code is working as intended. These iterative changes can make the difference between projects delivering reliable results or being abandoned.

## Python scripts

## Using unit tests to ensure code correctness

## Sharing Python packages

Python packages can be shared with others through PyPI and GitHub. Both methods make it possible for others to install your package using Pip.

### Sharing through the Python Package Index

Python packages can be published to the official Python Package Index (PyPI) to make them easily accessible to users. Packages hosted there can be installed by just running `pip install [packagename]`, where `[packagename]` is the name of your package. For example, my package for analysis of free-recall data, Psifr, can be installed by running `pip install psifr`. If you have followed the directions in this course for setting up an installable package with a `pyproject.toml` file, you have already done most of the work necessary to host a package on PyPI. See the [Python Packaging User Guide](https://packaging.python.org/en/latest/tutorials/packaging-projects/) for details.

### Sharing through GitHub

Users can also install packages from GitHub. For example, to install Psifr from the latest code on GitHub:

```bash
pip install git+https://github.com/mortonne/psifr
```

Running the command above makes it so we can now run code from modules in the Psifr package. For example, this code will load some sample data and convert it to a Polars DataFrame. After installing Psifr, uncomment the code to run it. In Visual Studio Code, you can uncomment a block of code by highlighting the code and running `Edit > Toggle Line Comment`.

In [1]:
# import polars as pl
# from psifr import fr
# df = fr.sample_data("Morton2013")
# data = pl.DataFrame(fr.merge_free_recall(df))
# data.head()

See the Pip documentation page on [VCS support](https://pip.pypa.io/en/stable/topics/vcs-support/) for details.

## Sharing Jupyter notebooks

## Soliciting bug reports and improvements